In [1]:
using CompTime, InteractiveUtils, MacroTools

In [2]:
function pw_log_rec(n, x)
    if iszero(n)
        one(x)
    elseif isone(n)
        x
    elseif isodd(n)
        x * pw_log_rec(n - 1, x)
    else
        pw_log_rec(n ÷ 2, x * x)
    end
end

pw_log_rec (generic function with 1 method)

In [3]:
pw_log_rec(0, "Abc")

""

In [4]:
pw_log_rec(1, "Abc")

"Abc"

In [5]:
pw_log_rec(5, "Abc")

"AbcAbcAbcAbcAbc"

In [6]:
function pw_log_loop(n, x)
    r = one(x)

    while n > 0
        if isodd(n)
            r = r * x
        end
        n = n ÷ 2
        x = x * x
    end

    return r
end

pw_log_loop (generic function with 1 method)

In [7]:
pw_log_loop(0, "Abc")

""

In [8]:
pw_log_loop(1, "Abc")

"Abc"

In [9]:
pw_log_loop(5, "Abc")

"AbcAbcAbcAbcAbc"

In [10]:
@ct_enable function pw_log_ct(@ct(n), x)
    @ct(k = n)
    r = one(x)

    @ct_ctrl while k > 0
        @ct_ctrl if isodd(k)
            r = r * x
        end
        @ct(k = k ÷ 2)
        x = x * x
    end

    return r

end

runtime (generic function with 1 method)

In [11]:
pw_log_ct(Val{5}, "Abc")

"AbcAbcAbcAbcAbc"

In [12]:
runtime(pw_log_ct, 5, "Abc")

"AbcAbcAbcAbcAbc"

In [13]:
comptime(pw_log_ct, Val{5}, "Abc")

"AbcAbcAbcAbcAbc"

In [14]:
@code_typed pw_log_ct(Val{5}, 10)

CodeInfo(
1 ─ %1 = Base.mul_int(1, x)::Int64
│   %2 = Base.mul_int(x, x)::Int64
│   %3 = Base.mul_int(%2, %2)::Int64
│   %4 = Base.mul_int(%1, %3)::Int64
└──      return %4
) => Int64

In [15]:
@code_lowered runtime(pw_log_ct, 3, 10)

CodeInfo(
1 ─      x@_7 = x@_4
│        k = n
└──      r = Main.one(x@_7)
2 ┄ %4 = k > 0
└──      goto #6 if not %4
3 ─ %6 = Main.isodd(k)
└──      goto #5 if not %6
4 ─      r = r * x@_7
5 ┄      k = k ÷ 2
│        x@_7 = x@_7 * x@_7
└──      goto #2
6 ─      return r
)

In [16]:
debug(pw_log_ct, Val{5}, 10) |> Base.remove_linenums! |> MacroTools.flatten

quote
    5
    r = one(x)
    r = r * x
    2
    x = x * x
    nothing
    1
    x = x * x
    r = r * x
    0
    x = x * x
    return r
end